In [4]:
import os

while os.path.basename(os.path.dirname(os.path.abspath('.'))) != 'code':
    os.chdir('../')
    from debugging.coco_panoptic_converter import upsnet_panoptic_converter
from instanceseg.datasets import labels_table_cityscapes
from instanceseg.panoeval.utils import check_annotation
from instanceseg.utils import instance_utils
import pathlib
import glob
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import json

from instanceseg.ext.panopticapi.utils import rgb2id
from scripts import evaluate

In [2]:
def converter_main_upsnet(predictions_dir, groundtruth_dir, problem_config, out_dirs_root, overwrite=False):
    out_dirs = {}
    out_jsons = {}
    if not os.path.exists(out_dirs_root):
        os.makedirs(out_dirs_root)
    else:
        print(Warning('{} already exists'.format(out_dirs_root)))
    indirs = {'gt': groundtruth_dir, 'pred': predictions_dir}
    for label_type, in_dir in indirs.items():
        dir_basename = 'panoptic_conversion_{}'.format(label_type)
        out_dir = os.path.join(out_dirs_root, dir_basename)
        file_list = sorted(glob.glob(os.path.join(in_dir, '*.png')))
        out_file = os.path.abspath(os.path.expanduser(os.path.join(out_dir, '..', dir_basename + '.json')))
        upsnet_panoptic_converter(out_dir, out_file, labels_file_list=file_list,
                                  labels_table=problem_config.labels_table, overwrite=overwrite)
        out_dirs[label_type] = out_dir
        out_jsons[label_type] = out_file

    # Sanity check
    with open(out_jsons['gt'], 'r') as f:
        import json
        gt_json = json.load(f)
        gt_annotations = gt_json['annotations']
    with open(out_jsons['pred'], 'r') as f:
        pred_json = json.load(f)
        pred_annotations = pred_json['annotations']
    categories = {el['id']: el for el in gt_json['categories']}
    check_annotation(out_dirs, gt_annotations[0], pred_annotations[0], categories)
    print('Conversion successful.  Predictions and groundtruth written to {}'.format([v for k, v in out_dirs.items()]))
    return out_jsons, out_dirs

In [5]:
split = 'val'
upsnet_results_directory = '/home/adelgior/code/upsnet/output/upsnet/cityscapes/upsnet_resnet50_cityscapes_4gpu/' \
                           + split + '/results/'
upsnet_pano_path = pathlib.Path(upsnet_results_directory, 'pans_unified')

labels_table = labels_table_cityscapes.get_cityscapes_trainids_label_table_cocoform()
problem_config = instance_utils.InstanceProblemConfig(n_instances_by_semantic_id=[None for _ in labels_table],
                                                      labels_table=labels_table)
gt_folder = '/home/adelgior/code/upsnet/data/cityscapes/panoptic'
gt_json_file = os.path.join(upsnet_pano_path, 'gt.json')
old_pred_json_file = upsnet_pano_path / 'pred.json'
pred_folder = upsnet_pano_path / 'pan_2ch'
new_pred_json_file = upsnet_pano_path / 'pred_with_image_ids.json'

# Fix json format
gt_json_format = json.load(open(gt_json_file, 'rb'))
image_ids = [l['id'] for l in gt_json_format['images']]
file_names = [l['file_name'] for l in gt_json_format['images']]
pred_json_format = json.load(open(old_pred_json_file, 'rb'))
assert len(image_ids) == len(pred_json_format['annotations']), \
    '{} image ids; {} annotations'.format(len(image_ids), len(pred_json_format['annotations']))
assert os.path.join(pred_folder)
for i in range(len(pred_json_format['annotations'])):
    pred_json_format['annotations'][i]['image_id'] = image_ids[i]
    pred_json_format['annotations'][i]['file_name'] = file_names[i]
json.dump(pred_json_format, new_pred_json_file.open('w'))

pred_folder = upsnet_pano_path / 'pan_2ch'


In [6]:
#%%
evaluate.main_unwrapped(gt_json_file, new_pred_json_file, gt_folder, pred_folder, problem_config, iou_threshold=0.5,
                        overwrite=False)


Evaluating from /home/adelgior/code/upsnet/output/upsnet/cityscapes/upsnet_resnet50_cityscapes_4gpu/val/results/pans_unified/gt.json, /home/adelgior/code/upsnet/output/upsnet/cityscapes/upsnet_resnet50_cityscapes_4gpu/val/results/pans_unified/pred_with_image_ids.json
Evaluating from /home/adelgior/code/upsnet/data/cityscapes/panoptic, /home/adelgior/code/upsnet/output/upsnet/cityscapes/upsnet_resnet50_cityscapes_4gpu/val/results/pans_unified/pan_2ch
Evaluation panoptic segmentation metrics:
Ground truth:
	Segmentation folder: /home/adelgior/code/upsnet/data/cityscapes/panoptic
	JSON file: /home/adelgior/code/upsnet/output/upsnet/cityscapes/upsnet_resnet50_cityscapes_4gpu/val/results/pans_unified/gt.json
Prediction:
	Segmentation folder: /home/adelgior/code/upsnet/output/upsnet/cityscapes/upsnet_resnet50_cityscapes_4gpu/val/results/pans_unified/pan_2ch
	JSON file: /home/adelgior/code/upsnet/output/upsnet/cityscapes/upsnet_resnet50_cityscapes_4gpu/val/results/pans_unified/pred_with_image

Traceback (most recent call last):
  File "/home/adelgior/code/fcn-instances-pytorch/instanceseg/ext/panopticapi/utils.py", line 16, in wrapper
    return f(*args, **kwargs)
  File "/usr0/home/adelgior/code/experimental-instanceseg/instanceseg/ext/panopticapi/evaluation.py", line 98, in pq_compute_single_core
    with Image.open(os.path.join(pred_folder, pred_ann['file_name'])) as img:
  File "/home/adelgior/virtualenvs/pytorch4_python3/lib/python3.7/site-packages/PIL/Image.py", line 2652, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: '/home/adelgior/code/upsnet/output/upsnet/cityscapes/upsnet_resnet50_cityscapes_4gpu/val/results/pans_unified/pan_2ch/frankfurt_000000_000576_leftImg8bit.png'


Core: 31, 0 from 1 images processed
Caught exception in worker thread:
Caught exception in worker thread:
Caught exception in worker thread:


Traceback (most recent call last):


Caught exception in worker thread:
Caught exception in worker thread:


...

  File "/usr0/home/adelgior/code/experimental-instanceseg/instanceseg/ext/panopticapi/evaluation.py", line 98, in pq_compute_single_core
    with Image.open(os.path.join(pred_folder, pred_ann['file_name'])) as img:
  File "/usr0/home/adelgior/code/experimental-instanceseg/instanceseg/ext/panopticapi/evaluation.py", line 98, in pq_compute_single_core
    with Image.open(os.path.join(pred_folder, pred_ann['file_name'])) as img:
  File "/home/adelgior/virtualenvs/pytorch4_python3/lib/python3.7/site-packages/PIL/Image.py", line 2652, in open
    fp = builtins.open(filename, "rb")
  File "/usr0/home/adelgior/code/experimental-instanceseg/instanceseg/ext/panopticapi/evaluation.py", line 98, in pq_compute_single_core
    with Image.open(os.path.join(pred_folder, pred_ann['file_name'])) as img:
FileNotFoundError: [Errno 2] No such file or directory: '/home/adelgior/code/upsnet/output/upsnet/cityscapes/upsnet_resnet50_cityscapes_4gpu/val/results/pans_unified/pan_2ch/munster_000161_000019_leftIm

Caught exception in worker thread:


FileNotFoundError: [Errno 2] No such file or directory: '/home/adelgior/code/upsnet/output/upsnet/cityscapes/upsnet_resnet50_cityscapes_4gpu/val/results/pans_unified/pan_2ch/munster_000160_000019_leftImg8bit.png'


Caught exception in worker thread:
Caught exception in worker thread:


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

